In [98]:
import os
import glob
import numpy as np
import nibabel as nib

Selection of Subject

In [295]:
SUB = "Subject_42_AmMa"
ID = "01"

In [111]:
SUB = "Subject_53_MazLo"
ID = "02"

In [31]:
SUB = "Subject_21_CoNi"
ID = "03"

Configuration Filenames

In [296]:
src = "/Users/paolo/Datasets/APSS-ConnectBrain-2019/original"
out = "/Users/paolo/Datasets/APSS-ConnectBrain-2019"

dir = "%s/sub-%s" % (out, ID)

nii_struct = dir + '/Niftii/Structural'
nii_diff = dir + '/Niftii/Diffusion/Preprocess'

trk_diss = dir + '/TRK/Dissection'
trk_diss_trackvis = dir + '/TRK/Dissection/TrackVis'
trk_tract = dir + '/TRK/Tractography'
trk_roi = dir + '/TRK/ROI'

In [146]:
!mkdir -p {nii_struct}
!mkdir -p {nii_diff}

!mkdir -p {trk_diss}
!mkdir -p {trk_diss_trackvis}
!mkdir -p {trk_tract}
!mkdir -p {trk_roi}

Convert subject naming

In [147]:
import os

In [148]:
all_src = '/Users/paolo/Datasets/APSS-ConnectBrain-2019/sub-%s' % ID
sub_old = SUB
sub_new = 'sub-%s' % ID

In [149]:
for s in os.walk(all_src):
    for f in s[2]:
        f_src = os.path.join(s[0], f)
        f_out = f_src.replace(sub_old, sub_new)
        os.rename(f_src, f_out)

Fix path Trackvis Scene

In [291]:
scene_src = src
scene_out = out

In [292]:
scene_dic = {'01': ('All_DTI', 'All_CSD_and_FAT', 'PT_CSD'),
            '02': ('All_1',),
            '03': ('All',)}

In [293]:
dir_old = '/Users/SS/Documents/Tractome_Data/%s' % SUB
dir_new = '%s/sub-%s' % (out, ID)
sub_old = SUB
sub_new = 'sub-%s' % ID
all_scene = scene_dic[ID]

In [294]:
for scene in all_scene:
    scene_old = scene_src + '/' + sub_new + '/TRK/Dissection/TrackVis/' + scene + '.scene'
    scene_new = scene_out + '/' + sub_new + '/TRK/Dissection/TrackVis/' + scene + '.scene'
    with open(scene_old, "rt") as fin:
        with open(scene_new, "wt+") as fout:
            for line in fin:
                line = line.replace(dir_old, dir_new)
                line = line.replace(sub_old, sub_new)
                fout.write(line)

Copy files

In [154]:
import os, glob, shutil

In [155]:
T1_flirt_src = "%s/sub-%s/Niftii/Structural/sub-%s_flirt.nii.gz" % (src, ID, ID)
T1_flirt_out = "%s/sub-%s/Niftii/Structural/sub-%s_flirt.nii.gz" % (out, ID, ID)
shutil.copy(T1_flirt_src, T1_flirt_out)

'/Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Structural/sub-01_flirt.nii.gz'

In [156]:
dwi_src = "%s/sub-%s/Niftii/Diffusion/sub-%s_iso.nii.gz" % (src, ID, ID)
dwi_out = "%s/sub-%s/Niftii/Diffusion/sub-%s_iso.nii.gz" % (out, ID, ID)
shutil.copy(dwi_src, dwi_out)

bval_src = "%s/sub-%s/Niftii/Diffusion/sub-%s.bval" % (src, ID, ID)
bval_out = "%s/sub-%s/Niftii/Diffusion/sub-%s.bval" % (out, ID, ID)
shutil.copy(bval_src, bval_out)

bvec_src = "%s/sub-%s/Niftii/Diffusion/sub-%s.bvec" % (src, ID, ID)
bvec_out = "%s/sub-%s/Niftii/Diffusion/sub-%s.bvec" % (out, ID, ID)
shutil.copy(bvec_src, bvec_out)

'/Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Diffusion/sub-01.bvec'

In [157]:
roi_src = "%s/sub-%s/TRK/ROI/*" % (src, ID)
for roi_old in glob.glob(roi_src):
    roi_new = "%s/sub-%s/TRK/ROI/%s" % (out, ID, os.path.basename(roi_old))
    shutil.copy(roi, roi_new)

In [158]:
roi_src = "%s/sub-%s/TRK/Dissection/TrackVis/*.nii" % (src, ID)
for roi_old in glob.glob(roi_src):
    roi_new = "%s/sub-%s/TRK/Dissection/TrackVis/%s" % (out, ID, os.path.basename(roi_old))
    shutil.copy(roi, roi_new)

Fix Diffusion Mask

In [159]:
dwi_mask_src = "%s/sub-%s/Niftii/Diffusion/Preprocess/sub-%s_iso_mask.nii.gz" % (src, ID, ID)
dwi_mask_out = "%s/sub-%s/Niftii/Diffusion/Preprocess/sub-%s_iso_mask.nii.gz" % (out, ID, ID)

In [160]:
!maskfilter -force -npass 3 {dwi_mask_src} erode {dwi_mask_out}

maskfilter: [WARNING] existing output files will be overwritten
maskfilter: [100%] uncompressing image "/Users/paolo/Datasets/APSS-ConnectBrain-2019/sub-01/Niftii/Diffusion/Preprocess/sub-01_iso_mask.nii.gz"
maskfilter: [100%] applying erode filter to image sub-01_iso_mask.nii.gz
maskfilter: [100%] compressing image "/Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Diffusion/Preprocess/sub-01_iso_mask.nii.gz"


DTI Model Reconstruction

In [161]:
dwi = "%s/sub-%s/Niftii/Diffusion/sub-%s_iso.nii.gz" % (out, ID, ID)
bval = "%s/sub-%s/Niftii/Diffusion/sub-%s.bval" % (out, ID, ID)
bvec = "%s/sub-%s/Niftii/Diffusion/sub-%s.bvec" % (out, ID, ID)
mask = "%s/sub-%s/Niftii/Diffusion/Preprocess/sub-%s_iso_mask.nii.gz" % (out, ID, ID)
fa = "%s/sub-%s/Niftii/Diffusion/sub-%s_FA.nii.gz" % (out, ID, ID)
cfa_dpy = "%s/sub-%s/Niftii/Diffusion/sub-%s_Color_FA_dpy.nii.gz" % (out, ID, ID)

In [162]:
dti = '/Users/paolo/Software/miniconda3/pkgs/dipy-0.16.0-py37h917ab60_0/bin/dipy_fit_dti'

In [163]:
!python {dti} --force --save_metrics 'fa' 'rgb' --out_fa {fa} --out_rgb {cfa_dpy}  {dwi} {bval} {bvec} {mask}

INFO:Computing DTI metrics for /Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Diffusion/sub-01_iso.nii.gz
INFO:Tensor estimation...
INFO:DTI metrics saved in current directory


CSD Model Reconstruction

In [164]:
dwi = "%s/sub-%s/Niftii/Diffusion/sub-%s_iso.nii.gz" % (out, ID, ID)
bval = "%s/sub-%s/Niftii/Diffusion/sub-%s.bval" % (out, ID, ID)
bvec = "%s/sub-%s/Niftii/Diffusion/sub-%s.bvec" % (out, ID, ID)
mask = "%s/sub-%s/Niftii/Diffusion/Preprocess/sub-%s_iso_mask.nii.gz" % (out, ID, ID)
fa = "%s/sub-%s/Niftii/Diffusion/sub-%s_FA.nii.gz" % (out, ID, ID)
pam = "%s/sub-%s/Niftii/Diffusion/sub-%s_csd.pam5" % (out, ID, ID)

In [165]:
csd = '/Users/paolo/Software/miniconda3/pkgs/dipy-0.16.0-py37h917ab60_0/bin/dipy_fit_csd'

In [166]:
!python {csd} --force --out_pam {pam} {dwi} {bval} {bvec} {mask} 

INFO:The following output files are about to be overwritten.
INFO:shm.nii.gz
INFO:peaks_indices.nii.gz
INFO:gfa.nii.gz
INFO:Loading /Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Diffusion/sub-01_iso.nii.gz
50.0 0.0
INFO:Computing response function
INFO:Eigenvalues for the frf of the input data are :[0.00155052 0.00040763 0.00040763]
INFO:Ratio for smallest to largest eigen value is 0.26290074600220675
INFO:CSD computation started.
INFO:CSD computation completed.
INFO:Pam5 file saved in /Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Diffusion


Tracking Workflow

In [167]:
mask = "%s/sub-%s/Niftii/Diffusion/Preprocess/sub-%s_iso_mask.nii.gz" % (out, ID, ID)
fa = "%s/sub-%s/Niftii/Diffusion/sub-%s_FA.nii.gz" % (out, ID, ID)
pam = "%s/sub-%s/Niftii/Diffusion/sub-%s_csd.pam5" % (out, ID, ID)
trk = "%s/sub-%s/TRK/Tractography/sub-%s_1M_apss_noaff.trk" % (out, ID, ID)

In [168]:
det = '/Users/paolo/Software/miniconda3/pkgs/dipy-0.16.0-py37h917ab60_0/bin/dipy_track_local'

In [169]:
!python {det} {pam} {fa} {fa} --out_tractogram {trk}

INFO:Local tracking on /Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/Niftii/Diffusion/sub-01_csd.pam5
INFO:Deterministic direction getter strategy selected
INFO:classifier done
INFO:seeds done
INFO:LocalTracking initiated
INFO:Saved /Users/paolo/Datasets/APSS-ConnectBrain-2019/fixed_sub/sub-01/TRK/Tractography/sub-01_1M_apss_noaff.trk


Fix Tracking Workflow

In [297]:
from nibabel.orientations import aff2axcodes
from dipy.tracking.streamline import transform_streamlines

In [317]:
fa = "%s/sub-%s/Niftii/Diffusion/sub-%s_FA.nii.gz" % (out, ID, ID)
pam = "%s/sub-%s/Niftii/Diffusion/sub-%s_csd.pam5" % (out, ID, ID)
trk_noaff = "%s/sub-%s/TRK/Tractography/sub-%s_1M_apss_noaff.trk" % (out, ID, ID)
trk = "%s/sub-%s/TRK/Tractography/sub-%s_1M_apss.trk" % (out, ID, ID)

In [318]:
img = nib.load(fa)
header = nib.streamlines.trk.TrkFile.create_empty_header()
header['voxel_to_rasmm'] = img.affine.copy()
header['voxel_sizes'] = img.header.get_zooms()[:3]
header['dimensions'] = img.shape[:3]
header['voxel_order'] = "".join(aff2axcodes(img.affine))

In [319]:
tract = nib.streamlines.load(trk_noaff)

In [320]:
fix_streamlines = transform_streamlines(tract.streamlines, np.linalg.inv(img.affine))

In [321]:
tractogram = nib.streamlines.Tractogram(fix_streamlines,affine_to_rasmm=img.affine)
nib.streamlines.save(tractogram, trk, header=header)

DTI Tracking

In [326]:
from dipy.direction import DeterministicMaximumDirectionGetter
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking)
from dipy.tracking import utils

In [323]:
fa = "%s/sub-%s/Niftii/Diffusion/sub-%s_FA.nii.gz" % (out, ID, ID)
trk = "%s/sub-%s/TRK/Tractography/sub-%s_1M_apss_test.trk" % (out, ID, ID)

In [ ]:
fa_img = nib.load(fa)

In [ ]:
dg = DeterministicMaximumDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                             max_angle=30.,
                                                             sphere=default_sphere)

In [ ]:
stop = fa_img.get_data()
stopping_thr = 0.25
classifier = ThresholdTissueClassifier(stop, stopping_thr)

In [325]:
seed_mask = fa_img.get_data()
seed_density = 1
seeds = utils.seeds_from_mask(
                seed_mask,
                density=[seed_density, seed_density, seed_density],
                affine=fa_img.affine)

In [ ]:
streamlines = LocalTracking(dg, classifier, seeds, fa_img.affine, step_size=.5)

In [ ]:
header = nib.streamlines.trk.TrkFile.create_empty_header()
header['voxel_to_rasmm'] = fa_img.affine.copy()
header['voxel_sizes'] = fa_img.header.get_zooms()[:3]
header['dimensions'] = fa_img.shape[:3]
header['voxel_order'] = "".join(aff2axcodes(fa_img.affine))

In [ ]:
tractogram = nib.streamlines.Tractogram(streamlines,affine_to_rasmm=np.eye(4))
nib.streamlines.save(tractogram, trk, header=header)

Convert Color FA for Trackvis

In [141]:
def decfa(img_orig, scale=False):
    """
    Create a nifti-compliant directional-encoded color FA image.
    Parameters
    ----------
    img_orig : Nifti1Image class instance.
        Contains encoding of the DEC FA image with a 4D volume of data, where
        the elements on the last dimension represent R, G and B components.
    scale: bool.
        Whether to scale the incoming data from the 0-1 to the 0-255 range
        expected in the output.
    Returns
    -------
    img : Nifti1Image class instance with dtype set to store tuples of
        uint8 in (R, G, B) order.
    Notes
    -----
    For a description of this format, see:
    https://nifti.nimh.nih.gov/nifti-1/documentation/nifti1fields/nifti1fields_pages/datatype.html
    """

    dest_dtype = np.dtype([('R', 'uint8'), ('G', 'uint8'), ('B', 'uint8')])
    out_data = np.zeros(img_orig.shape[:3], dtype=dest_dtype)

    data_orig = img_orig.get_data()

    if scale:
        data_orig = (data_orig * 255).astype('uint8')

    for ii in np.ndindex(img_orig.shape[:3]):
        val = data_orig[ii]
        out_data[ii] = (val[0], val[1], val[2])

    new_hdr = img_orig.header
    new_hdr['dim'][4] = 1
    new_hdr.set_intent(1001, name='Color FA')
    new_hdr.set_data_dtype(dest_dtype)
    
    return nib.Nifti1Image(out_data, affine=img_orig.affine, header=new_hdr)

In [180]:
cfa_dpy = "%s/sub-%s/Niftii/Diffusion/sub-%s_Color_FA_dpy.nii.gz" % (out, ID, ID)
cfa_trk = "%s/sub-%s/Niftii/Diffusion/sub-%s_Color_FA_trk.nii.gz" % (out, ID, ID)

In [ ]:
img = nib.load(cfa_dpy)
img_fix = decfa(img)
nib.save(img_fix, cfa_trk)

Cleaning tractogram

In [280]:
trk = "/Users/paolo/Downloads/slf_test.trk" 
trk_clean = '/Users/paolo/Downloads/slf_clean.trk' 
trk_del = '/Users/paolo/Downloads/slf_del.trk'

In [281]:
t_src = nib.streamlines.load(trk)
t_del = nib.streamlines.load(trk_del)

In [286]:
s_clean = []
for s_src in t_src.streamlines:
    s_add = True
    for s_del in t_del.streamlines:
        if np.array_equiv(s_del, s_src):
            s_add = False
            print('DEL')
            break
    if s_add:
        s_clean.append(s_src)

DEL
DEL
DEL
DEL
DEL
DEL
DEL
DEL
DEL


In [284]:
t_clean = nib.streamlines.Tractogram(s_clean,affine_to_rasmm=np.eye(4))
nib.streamlines.save(t_clean, trk_clean, header=t_src.header)

In [253]:
len(s_clean)

102180

In [250]:
len(t_src.streamlines)

102180

In [255]:
len(t_del.streamlines)

14

In [275]:
a1 = np.array([[1,1],[2,2]])
a2 = np.array([[1,0.9999999],[2,2]])

In [276]:
np.array_equiv(a1,a2)

False

In [273]:
(a1==a2).all()

True

Affine Registration

In [261]:
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)

In [262]:
def affreg(data_A, aff_A, data_B, aff_B):

    # Center of mass alignment
    c_of_mass = transform_centers_of_mass(data_A, aff_A, data_B, aff_B)
    
    #data_B2A = c_of_mass.transform(data_B)
    
    # Affine registration: Translation Transformation

    affreg = AffineRegistration()

    # Affine registration: Translation Transformation

    transform = TranslationTransform3D()
    translation = affreg.optimize(data_A, data_B, transform, None,
                                  aff_A, aff_B,
                                  starting_affine=c_of_mass.affine)
    #data_B2A = translation.transform(data_B)


    # Affine registration: Rigid Transformation

    transform = RigidTransform3D()
    rigid = affreg.optimize(data_A, data_B, transform, None,
                            aff_A, aff_B,
                            starting_affine=translation.affine)
    #data_B2A = rigid.transform(b0_mask_B)

    # Affine registration: final step

    transform = AffineTransform3D()
    affine = affreg.optimize(data_A, data_B, transform, None,
                             aff_A, aff_B,
                             starting_affine=rigid.affine)
    #b0_mask_B = affine.transform(b0_mask_B)
    
    return affine.transform(data_B), affine.affine

In [309]:
fa_A = src + 'tmp/sub-01_FA.nii.gz'
fa_B = src + '../sub-03/Niftii/Diffusion/Subject_21_CoNi_FA.nii.gz'
fa_B2A = src + 'tmp/sub-01_FA_b2a.nii.gz'

In [310]:
img_A =nib.load(fa_A)
data_A = img_A.get_data()
aff_A = img_A.affine

img_B =nib.load(fa_B)
data_B = img_B.get_data()
aff_B = img_B.affine

In [311]:
data_B2A, aff_B2A = affreg(data_A, aff_A, data_B, aff_B)

Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]


In [324]:
map_B2A = AffineMap(aff_B2A, data_A.shape, aff_A, data_B.shape, aff_B)
data_test = map_B2A.transform(data_B)

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

data_A = nib.load(fa_A).get_data()
sli = data_A.shape[2] // 2
plt.figure('Registration')
plt.subplot(1, 4, 1).set_axis_off()
plt.imshow(histeq(data_A[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

data_B = nib.load(fa_B).get_data()
sli = data_B.shape[2] // 2
plt.subplot(1, 4, 2).set_axis_off()
plt.imshow(histeq(data_B[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

sli = data_B2A.shape[2] // 2
plt.subplot(1, 4, 3).set_axis_off()
plt.imshow(histeq(data_B2A[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

sli = data_test.shape[2] // 2
plt.subplot(1, 4, 4).set_axis_off()
plt.imshow(histeq(data_test[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

___

(uncomplete) Pipeline from scratch

In [ ]:
dcm2nii_tmp = tmp + '/dcm2nii'
!mkdir -p {tmp}
!mkdir -p {dcm2nii_tmp}

Convert Structural DICOM

In [ ]:
!dcm2nii -d n -g y -s n -p n -o {dcm2nii_tmp} {dicom_struct} 

In [ ]:
T1_src = glob.glob(dcm2nii_tmp + '/co*.nii.gz')[0]
T1_out = '%s/sub-%s_T1.nii.gz' % (nii_struct, ID)

In [ ]:
!mv {T1_src} {T1_out}
!rm {dcm2nii_tmp}/*

Convert Diffusion DICOM

In [ ]:
!dcm2nii -d n -g y -s n -p n -o {dcm2nii_tmp} {dicom_diff} 

In [ ]:
dwi_src = glob.glob(dcm2nii_tmp + '/*.nii.gz')[0]
dwi_out = '%s/sub-%s_dwi.nii.gz' % (nii_diff, ID)
bval_src = glob.glob(dcm2nii_tmp + '/*.bval')[0]
bval_out = '%s/sub-%s_dwi.bval' % (nii_diff, ID)
bvec_src = glob.glob(dcm2nii_tmp + '/*.bvec')[0]
bvec_out = '%s/sub-%s_dwi.bvec' % (nii_diff, ID)

In [ ]:
!mv {dwi_src} {dwi_out}
!mv {bval_src} {bval_out}
!mv {bvec_src} {bvec_out}

Structural Brain Extraction

In [ ]:
from deepbrain import Extractor

In [ ]:
T1_src = '%s/sub-%s_T1.nii.gz' % (nii_struct, ID)
T1_mask_out = '%s/sub-%s_T1_mask.nii.gz' % (nii_struct, ID)
T1_brain_out = '%s/sub-%s_T1_brain.nii.gz' % (nii_struct, ID)

In [ ]:
img = nib.load(T1_src)
data = img.get_fdata()
affine = img.affine

ext = Extractor()
prob = ext.run(data)
mask = prob > 0.05
brain = data[:]
brain[~mask] = 0

In [ ]:
img = nib.Nifti1Image(mask.astype(np.int), affine)
nib.save(img, T1_mask_out)
img = nib.Nifti1Image(brain, affine)
nib.save(img, T1_brain_out)

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

sli = brain.shape[2] // 2
plt.figure('Brain segmentation')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(histeq(brain[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(histeq(mask[:, :, sli].astype('int')).T,
           cmap='gray', origin='lower')

In [ ]:
T1_src = '%s/sub-%s_T1.nii.gz' % (nii_struct, ID)
T1_mask_out = '%s/sub-%s_T1_bet_mask.nii.gz' % (nii_struct, ID)
T1_brain_out = '%s/sub-%s_T1_bet.nii.gz' % (nii_struct, ID)

In [ ]:
!bet {T1_src} {T1_brain_out}  -F -f .2 -g 0

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

img = nib.load(T1_brain_out)
bet = img.get_data()
img = nib.load(T1_mask_out)
mask = img.get_data()

sli = bet.shape[2] // 2
plt.figure('Brain segmentation')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(histeq(bet[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(histeq(mask[:, :, sli].astype('int')).T,
           cmap='gray', origin='lower')

Diffusion Brain Extraction

In [ ]:
from dipy.segment.mask import median_otsu

In [ ]:
dwi_src = '%s/sub-%s_dwi.nii.gz' % (nii_diff, ID)
bval_src = '%s/sub-%s_dwi.bval' % (nii_diff, ID)
bvec_src = '%s/sub-%s_dwi.bvec' % (nii_diff, ID)

b0_out = '%s/sub-%s_b0.nii.gz'% (nii_diff, ID)
dwi_mask_out = '%s/sub-%s_dwi_mask.nii.gz'% (nii_diff, ID)

In [ ]:
img = nib.load(dwi_src)
data = img.get_data()
b0 = data[:,:,:,0]
affine = img.affine

In [ ]:
b0_mask, mask = median_otsu(b0, 2, 1)

In [ ]:
mask_img = nib.Nifti1Image(mask.astype(np.float32), affine)
b0_img = nib.Nifti1Image(b0_mask.astype(np.float32), affine)

nib.save(mask_img, dwi_mask_out)
nib.save(b0_img, b0_out)

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

sli = b0.shape[2] // 2
plt.figure('Brain segmentation')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(histeq(b0[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(histeq(b0_mask[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

#plt.savefig('median_otsu.png')

Isotropic Diffusion

In [ ]:
from dipy.align.reslice import reslice

In [ ]:
dwi_src = '%s/sub-%s_dwi.nii.gz' % (nii_diff, ID)
dwi_iso = '%s/sub-%s_dwi_iso.nii.gz' % (nii_diff, ID)

img = nib.load(dwi_src)
data = img.get_data()
affine = img.affine
size_src = img.header.get_zooms()[:3]
size_iso = np.array([2.0, 2.0, 2.0])

In [ ]:
data_iso, affine_iso = reslice(data, affine, size_src, size_iso)

In [ ]:
img_iso = nib.Nifti1Image(data_iso, affine_iso)
nib.save(img_iso, dwi_iso)

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

data = data[...,0]
sli = data.shape[2] // 2
plt.figure('Isotropic diffusion')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(histeq(data[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

data_iso = data_iso[...,0]
sli = data_iso.shape[2] // 2
plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(histeq(data_iso[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

Denoise diffusion

In [ ]:
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma

In [ ]:
dwi_src = '%s/sub-%s_dwi_iso.nii.gz' % (nii_diff, ID)
dwi_mask = '%s/sub-%s_dwi_mask.nii.gz'% (nii_diff, ID)
dwi_den = '%s/sub-%s_dwi_den.nii.gz' % (nii_diff, ID)
bval_src = '%s/sub-%s_dwi.bval' % (nii_diff, ID)
bvec_src = '%s/sub-%s_dwi.bvec' % (nii_diff, ID)

In [ ]:
img = nib.load(dwi_src)
data = img.get_data()
affine = img.affine

img = nib.load(dwi_mask)
mask = img.get_data()

In [ ]:
sigma = estimate_sigma(data, N=4)
den = nlmeans(data, sigma=sigma, mask=mask, patch_radius= 1, block_radius = 1, rician= True)

In [ ]:
img_den = nib.Nifti1Image(den, affine)
nib.save(img_den, dwi_den)

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

b0 = data[...,0]
sli = b0.shape[2] // 2
plt.figure('Isotropic diffusion')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(histeq(b0[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

b0_den = den[...,0]
sli = b0_den.shape[2] // 2
plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(histeq(b0_den[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

White Matter Mask

In [ ]:
md = '%s/sub-%s_md.nii.gz' % (nii_diff, ID)
fa = '%s/sub-%s_fa.nii.gz' % (nii_diff, ID)
wm = '%s/sub-%s_wm.nii.gz' % (nii_diff, ID)

In [ ]:
img = nib.load(md)
MD = img.get_data()
img = nib.load(fa)
FA = img.get_data()
affine = img.affine

In [ ]:
WM = (np.logical_or(FA >= 0.4, (np.logical_and(FA >= 0.15, MD >= 0.0011))))
WM = WM.astype('uint16')

In [ ]:
img_wm = nib.Nifti1Image(WM, affine)
nib.save(img_wm, wm)

In [ ]:
import matplotlib.pyplot as plt
from dipy.core.histeq import histeq

sli = FA.shape[2] // 2
plt.figure('Isotropic diffusion')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(histeq(FA[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

sli = WM.shape[2] // 2
plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(histeq(WM[:, :, sli].astype('float')).T,
           cmap='gray', origin='lower')

Registration of T1 to Diffusion

Registration of Atlas